In [1]:
# REQUIREMENTS
! pip install ipyleaflet
! pip install bs4
! pip install requests


In [2]:
#Import libraries
from ipyleaflet import Map, basemaps, basemap_to_tiles, TileLayer, Marker, MarkerCluster, VectorTileLayer
from bs4 import BeautifulSoup
import requests


### Scrape all available URLS using Beautiful Soup


In [3]:
#This URL can be changed
server_url='https://mapcore-demo.org/current/flatmap/v2'
req= requests.get(server_url)
url_dict = req.json()

In [4]:
#JOB : There are fields with non IDs, len=6 eliminated this, think of a smarter way!
model_ids = [model_dict['id'] for model_dict in url_dict if 'name' in model_dict.keys() and 'id' in model_dict.keys()]
model_name = [model_dict['name'] for model_dict in url_dict if 'name' in model_dict.keys() and 'id' in model_dict.keys()]
  

### Get all layers for each model to be added sequentially to the map

In [5]:
model_layer_url_list = [f'{server_url}/flatmap/{id}/layers' for id, name in zip(model_ids, model_name)]

In [6]:
model_image_layers = []
for url in model_layer_url_list:
  req= requests.get(url)
  layer_url_dict = req.json()
  model_image_layers.append({layer_url_dict[0]['id'] : layer_url_dict[0]['image-layers']})

In [7]:
model_image_layers

[{'vagus_test': ['vagus_test_image']},
 {'whole-human': ['whole-human_image']},
 {'whole-rat': ['whole-rat_image',
   'whole-rat_details_vagus_image',
   'whole-rat_details_tissue-slide_image']},
 {'whole-pig': ['whole-pig_image']},
 {'whole-rat': ['whole-rat_image',
   'whole-rat_details_vagus_image',
   'whole-rat_details_tissue-slide_image']},
 {'whole-rat': ['whole-rat_image', 'whole-rat_details_tissue-slide_image']},
 {'whole-rat': ['whole-rat_image',
   'whole-rat_details_vagus_image',
   'whole-rat_details_tissue-slide_image']},
 {'whole-rat': ['whole-rat_image',
   'whole-rat_details_vagus_image',
   'whole-rat_details_tissue-slide_image']},
 {'whole-mouse': ['whole-mouse_image']},
 {'whole-cat': ['whole-cat_image']},
 {'whole-human': ['whole-human_image']},
 {'whole-rat': ['whole-rat_image',
   'whole-rat_details_vagus_image',
   'whole-rat_details_tissue-slide_image']}]

### Create a map with all layers of a model added seqentially to a map

In [8]:
#Choose one of the models in tag (There are 11 in the main URL)
tag = 0
tile_urls = [f'{server_url}/flatmap/{model_ids[tag]}/tiles/{layer}/{{z}}/{{x}}/{{y}}' for layer in model_image_layers[tag][model_name[tag]]]
vector_tile_url = f'{server_url}/flatmap/{model_ids[tag]}/mvtiles/{{z}}/{{x}}/{{y}}'

In [9]:
req = requests.get(f'{server_url}/flatmap/{model_ids[tag]}',headers={'Accept':'json'})
index_json = req.json()

In [10]:
#Create a base map object with the first tile
map = Map(
    basemap=TileLayer(
                    url=tile_urls[0], 
                    min_zoom=index_json['min-zoom']),
    min_zoom=index_json['min-zoom'],
    max_zoom=index_json['max-zoom'],
    zoom=5,
    scroll_wheel_zoom=True,
    dragging=True,
    attribution_control=False,
    zoom_snap=False,
)

#Create tile objects for the model with each layer added
for tile_url in tile_urls[1:]:
  map.add(TileLayer(
      url=tile_url, 
      min_zoom=index_json['min-zoom']))

# Add vector tile layer
map.add_layer(VectorTileLayer(url=vector_tile_url))

bounds = index_json['bounds']
map.fit_bounds([[bounds[1],bounds[0]],[bounds[3],bounds[2]]])

### Add annotations for the map

In [11]:
# Get annotations for this map
req = requests.get(f'{server_url}/flatmap/{model_ids[tag]}/annotations')
annotations = req.json()

In [12]:
annotations

{'1': {'bounds': [-2.2738391623736414,
   -0.6481569112071363,
   -1.4317597174753078,
   0.6422536904945128],
  'centroid': [-1.8676237507285933, 0.044769731516285764],
  'featureId': 1,
  'geometry': 'Polygon',
  'id': 'body_5',
  'label': 'ear',
  'layer': 'vagus_test',
  'markup': '.id(body_5)',
  'models': 'UBERON:0001690',
  'tile-layer': 'features'},
 '11': {'bounds': [0.49031650020317546,
   -1.2704008289497488,
   1.7116157614647753,
   1.2048736922621064],
  'centroid': [1.0347929466469483, -0.0010293068995796567],
  'featureId': 11,
  'geometry': 'LineString',
  'layer': 'vagus_test',
  'tile-layer': 'features'},
 '12': {'bounds': [1.216513716825076,
   -0.991609180327596,
   2.121341517259878,
   1.2333260131815722],
  'centroid': [1.765491995442746, 0.16584468782030035],
  'featureId': 12,
  'geometry': 'LineString',
  'layer': 'vagus_test',
  'tile-layer': 'features'},
 '16': {'bounds': [0.49031650020317546,
   -1.2704008289497488,
   2.12135007371296,
   1.23331745871072

In [13]:
# Add all annotations to a Marker Cluster
markers = []
for key in annotations.keys():
    y,x = annotations[key]['centroid']
    if 'label' in annotations[key].keys():
        marker = Marker(location=[x,y],title=annotations[key]['label'])
    else:
        marker = Marker(location=[x,y])
    markers.append(marker)

map.add_layer(MarkerCluster(markers=markers))

In [14]:
map

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…